In [1]:
import pandas as pd
import numpy as np

In [2]:
master_path = r"..\Master-0015442-FS-8100-BOM-20200326.xlsx"
compared_path = r"..\0015442-FS-8100-BOM-20200326.xlsx"

In [3]:
master_df = pd.read_excel(master_path)
compared_df = pd.read_excel(compared_path)

In [4]:
master_df.index.set_names('Order', inplace=True)
compared_df.index.set_names('Order', inplace=True)

In [5]:
master_df.at[1, 'File Name']

'    0019043.prt'

In [6]:
def id_immediate_parent(df_child_row, df):
    immediate_parent_level = -1
    immediate_parent_index = -1
    immediate_parent_file_name = ''

    if not df_child_row['Level'] == 0:
        # get parent level
        immediate_parent_level = df_child_row['Level'] - 1

        # get parent index
        immediate_parent_index = df['Level'].iloc[:df_child_row.name].eq(immediate_parent_level).sort_index(ascending=False).idxmax()

        # get parent file name
        immediate_parent_file_name = df.at[immediate_parent_index, 'File Name'].strip()

    return [df_child_row.name, immediate_parent_index, immediate_parent_level, immediate_parent_file_name]

In [7]:
immediate_parent_id_df = master_df.apply(func=id_immediate_parent, axis=1, result_type='expand', args=(master_df, ))

In [8]:
immediate_parent_id_df.columns = ['Child Index', 'Last Parent Index', 'Last Parent Level', 'Last Parent File Name']

In [9]:
immediate_parent_id_df.set_index('Child Index', inplace=True)

In [10]:
immediate_parent_id_df

,Last Parent Index,Last Parent Level,Last Parent File Name
Child Index,,,
0,-1,-1,
1,0,0,0015442.asm
2,0,0,0015442.asm
3,2,1,0018435.asm
4,3,2,0018442.asm
...,...,...,...
817,0,0,0015442.asm
818,817,1,0018168.asm
819,817,1,0018168.asm


In [11]:
def generate_immediate_parent_id_df(df):
    immediate_parent_id_df = df.apply(func=id_immediate_parent, axis=1, result_type='expand', args=(df, ))
    immediate_parent_id_df.columns = ['Order', 'Immediate Parent Index', 'Immediate Parent Level', 'Immediate Parent File Name']
    return immediate_parent_id_df.set_index('Order')

In [12]:
def crawl_immediate_parents(child_row, immediate_parent_id_df):
    if child_row['Immediate Parent Level'] == -1:
        return [], [], []
    else:
        parent_row = immediate_parent_id_df.loc[child_row['Immediate Parent Index']]

        parent_indices, parent_levels, parent_file_names = crawl_immediate_parents(parent_row, immediate_parent_id_df)

        parent_indices.extend([child_row['Immediate Parent Index']])
        parent_levels.extend([child_row['Immediate Parent Level']])
        parent_file_names.extend([child_row['Immediate Parent File Name']])
        
        return parent_indices, parent_levels, parent_file_names

In [13]:
def generate_all_parent_id_df(df):
    immediate_parent_id_df = generate_immediate_parent_id_df(df)
    all_parent_id_df = immediate_parent_id_df.apply(crawl_immediate_parents, axis=1, result_type='expand', args=(immediate_parent_id_df, ))
    all_parent_id_df.columns = ['Parent Indices', 'Parent Levels', 'Parent File Names']
    
    return all_parent_id_df

In [14]:
def id_parents(df):
    return generate_all_parent_id_df(df)['Parent File Names'].str.join('->')

In [15]:
def id_nodes(df):
    return id_parents(df).str.cat(df['File Name'].str.strip(), sep="->")

In [16]:
master_path = r"..\ShortMaster-0015442-FS-8100-BOM-20200326.xlsx"
compared_path = r"..\ShortTest-0015442-FS-8100-BOM-20200326.xlsx"
master_df = pd.read_excel(master_path, keep_default_na=False)
compared_df = pd.read_excel(compared_path, keep_default_na=False)
master_df.index.set_names('Order', inplace=True)
compared_df.index.set_names('Order', inplace=True)

In [17]:
compare_columns = [master_column for master_column in master_df.columns if master_column in compared_df.columns]

In [18]:
master_df['Node ID'] = id_nodes(master_df)
compared_df['Node ID'] = id_nodes(compared_df)

In [19]:
# ~master_df['Node ID'].isin(compared_df['Node ID']).values

In [20]:
master_df.set_index('Node ID', append=True, inplace=True)
compared_df.set_index('Node ID', append=True, inplace=True)

In [21]:
deleted_nodes = ~master_df.index.get_level_values('Node ID').isin(compared_df.index.get_level_values('Node ID'))

In [22]:
# deleted_nodes

In [23]:
# ~master_df.reset_index()['Node ID'].isin(compared_df.reset_index()['Node ID'])

In [24]:
added_nodes = ~compared_df.index.get_level_values('Node ID').isin(master_df.index.get_level_values('Node ID'))

In [25]:
updated_elements = ~master_df.loc[slice(None), compare_columns].reset_index('Order').eq(compared_df[slice(None)].reset_index('Order'))

In [26]:
# master_df[compare_columns].reset_index('Order').merge(compared_df.reset_index('Order'), left_on='Node ID', right_on='Node ID', how='outer', indicator=True)

In [27]:
# updated_elements

In [28]:
# master_df[deleted_nodes]

In [29]:
compared_df[added_nodes]

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,TestOnly
Order,Node ID,,,,,,,,,,
12,0015442.asm->0018435.asm->0019012.asm,2,0019012.asm,"HAT, ICE BIN DRAIN TUBE ASSY","HAT, ICE BIN DRAIN TUBE ASSY",,"HAT, ICE BIN DRAIN TUBE ASSY",1,0-00.1,Processing,
13,0015442.asm->0018435.asm->0019012.asm->0018439_revised_02.prt,3,0018439_revised_02.prt,0018439_revised_02.prt,0018439_revised_02.prt,,0018439_revised_02.prt,1,0-00.1,Processing,
14,0015442.asm->0018435.asm->0019012.asm->0019011.prt,3,0019011.prt,"HAT, ICE BIN DRAIN TUBE","HAT, ICE BIN DRAIN TUBE",,"HAT, ICE BIN DRAIN TUBE",1,0-00.1,Processing,
15,0015442.asm->0018435.asm->0019012.asm->0007471.prt,3,0007471.prt,"RIVET, BD, 0.125"" DIA, 0.063""-0.125"" RNG, 420L...","RIVET, BLIND .125 DIA X .125, SS, DOME HEAD",,"RIVET, BLIND .125 DIA X .125, SS, DOME HEAD",8,0-00.0,Released,
16,0015442.asm->0018435.asm->0019012.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,,RIVET,4,0-00.2,Released,
17,0015442.asm->0012397.asm,1,0012397.asm,"ASSM, 9100 LEG",0012397.asm,,0012397.asm,2,0-02.0,Released,
18,0015442.asm->0012397.asm->0012398.prt,2,0012398.prt,0012398.prt,0012398.prt,,0012398.prt,2,0-02.0,Released,


In [145]:
deleted_nodes

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False])

In [30]:
compared_df.loc[added_nodes, 'File Name'].isin(master_df['File Name'].values)

Order  Node ID                                                      
12     0015442.asm->0018435.asm->0019012.asm                            False
13     0015442.asm->0018435.asm->0019012.asm->0018439_revised_02.prt    False
14     0015442.asm->0018435.asm->0019012.asm->0019011.prt               False
15     0015442.asm->0018435.asm->0019012.asm->0007471.prt               False
16     0015442.asm->0018435.asm->0019012.asm->0012468.prt                True
17     0015442.asm->0012397.asm                                         False
18     0015442.asm->0012397.asm->0012398.prt                            False
Name: File Name, dtype: bool

In [31]:
import id_changes
import importlib

In [32]:
importlib.reload(id_changes)

<module 'id_changes' from 'c:\\Users\\Luke Smith\\Desktop\\Coca-Cola\\BOM Mgmt\\bom-checker\\id_changes.py'>

In [33]:
updated_elements = id_changes.get_updated_elements(master_df, compared_df, order_only=None)

In [34]:
updated_elements

,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State
Node ID,,,,,,,,,
->0015442.asm,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0014175.prt,False,False,False,False,False,False,False,True,False
0015442.asm->0018435.asm->0018436.prt,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0018437.prt,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0018438.prt,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0018442.asm,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0018442.asm->0011799.prt,False,False,False,False,False,False,False,False,False
0015442.asm->0018435.asm->0018442.asm->0012468.prt,False,False,False,False,False,False,False,False,False


In [35]:
# master_df.reset_index().set_index('Node ID')[updated_elements].combine_first(master_df.reset_index().set_index('Node ID'))

In [36]:
support_columns = id_changes.get_support_columns(master_df, compared_df)

In [37]:
updated_bom_df = compared_df.merge(master_df.loc[~deleted_nodes, support_columns], on='Node ID', how='outer')

In [127]:
def highlight(element, color='yellow'):
    return 'background-color: {}'.format(color)

In [128]:
def highlight_true(element, color='yellow'):
    if not element == True:
        color = 'none'
        
    return highlight(element, color=color)

In [129]:
def highlight_valid(element, color='yellow'):
    while True:
        try:
            element_valid = not element.lower() == 'nan'
            break
        except:
            pass

        try:
            element_valid = not np.isnan(element)
            break
        except:
            pass
        
        element_valid = element
        break
        
    if not element_valid:
        color = 'none'

    return highlight(element, color=color)

In [130]:
def highlight_updated(column, updated_elements, color='yellow'):
    nodes = column.index.get_level_values('Node ID')
    return [highlight_true(element) for element in updated_elements.loc[nodes, column.name]]

In [111]:
master_df.loc[~deleted_nodes, compare_columns].style.apply(highlight_updated, axis=0, args=(updated_elements, ))

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State
Order,Node ID,,,,,,,,,
0,->0015442.asm,0,0015442.asm,0015442.asm,"SHIPPING CONFIG, GS2",-,"SHIPPING CONFIG, GS2",,0-00.50,Processing
1,0015442.asm->0019043.prt,1,0019043.prt,FS-8100 ID SURFACE MODEL,FS-8100 ID SURFACE MODEL,-,FS-8100 ID SURFACE MODEL,1,0-00.37,Processing
2,0015442.asm->0018435.asm,1,0018435.asm,FS-8100 Chassis assembly - full,FS-8100 Chassis assembly - full,,FS-8100 Chassis assembly - full,1,0-00.24,Processing
3,0015442.asm->0018435.asm->0018442.asm,2,0018442.asm,FS-8100 Chassis Base Assembly,FS-8100 Chassis Base Assembly,,FS-8100 Chassis Base Assembly,1,0-00.6,Processing
4,0015442.asm->0018435.asm->0018442.asm->0018441.prt,3,0018441.prt,FS-9100 base blate - btm plate,FS-9100 base blate - btm plate,,FS-9100 base blate - btm plate,1,0-00.7,Processing
6,0015442.asm->0018435.asm->0018442.asm->0011799.prt,3,0011799.prt,"STUD, SC-FH, M6X1.0, 18MM LG, 1.6MM MIN THK, SS 300, PSVT","STUD,SELF-CLINCHING,THD,M6-18,FHS",,"STUD,SELF-CLINCHING,THD,M6-18,FHS",12,0-00.4,Alpha Prototype
7,0015442.asm->0018435.asm->0018442.asm->0014161.prt,3,0014161.prt,"3 PROJECTION WELDNUT, 3/4""-10","3 PROJECTION WELDNUT, 3/4""-10",,"3 PROJECTION WELDNUT, 3/4""-10",4,0-00.1,Released
8,0015442.asm->0018435.asm->0018442.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,,RIVET,4,0-00.2,Released
9,0015442.asm->0018435.asm->0018436.prt,2,0018436.prt,FS-8100 Chassis vert member SM-Box,FS-8100 Chassis vert member SM-Box,,FS-8100 Chassis vert member SM-Box,4,0-00.15,Processing


In [112]:
updated_bom_df[updated_elements].style.applymap(highlight_valid)

,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,TestOnly,AddedColumn1,AddedColumn2,AddedColumn3,AddedColumn4,AddedColumn5,AddedColumn6
Node ID,,,,,,,,,,,,,,,,
->0015442.asm,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0019043.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018442.asm,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018442.asm->0018441.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018442.asm->0011799.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018442.asm->0014161.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018442.asm->0012468.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0018436.prt,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [135]:
def highlight_names(iterable, names, color='yellow'):
    if not iterable.name in names:
        color = 'none'
    
    return [highlight(element, color=color) for element in iterable]

In [136]:
updated_bom_df[added_nodes].style.apply(highlight_names, axis=0, args=(['Level', 'File Name'], 'blue'))

,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,TestOnly,AddedColumn1,AddedColumn2,AddedColumn3,AddedColumn4,AddedColumn5,AddedColumn6
Node ID,,,,,,,,,,,,,,,,
0015442.asm->0018435.asm->0019012.asm,2,0019012.asm,"HAT, ICE BIN DRAIN TUBE ASSY","HAT, ICE BIN DRAIN TUBE ASSY",,"HAT, ICE BIN DRAIN TUBE ASSY",1,0-00.1,Processing,,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0019012.asm->0018439_revised_02.prt,3,0018439_revised_02.prt,0018439_revised_02.prt,0018439_revised_02.prt,,0018439_revised_02.prt,1,0-00.1,Processing,,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0019012.asm->0019011.prt,3,0019011.prt,"HAT, ICE BIN DRAIN TUBE","HAT, ICE BIN DRAIN TUBE",,"HAT, ICE BIN DRAIN TUBE",1,0-00.1,Processing,,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0019012.asm->0007471.prt,3,0007471.prt,"RIVET, BD, 0.125"" DIA, 0.063""-0.125"" RNG, 420LBS SHEAR, SS","RIVET, BLIND .125 DIA X .125, SS, DOME HEAD",,"RIVET, BLIND .125 DIA X .125, SS, DOME HEAD",8,0-00.0,Released,,nan,nan,nan,nan,nan,nan
0015442.asm->0018435.asm->0019012.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,,RIVET,4,0-00.2,Released,,nan,nan,nan,nan,nan,nan
0015442.asm->0012397.asm,1,0012397.asm,"ASSM, 9100 LEG",0012397.asm,,0012397.asm,2,0-02.0,Released,,nan,nan,nan,nan,nan,nan
0015442.asm->0012397.asm->0012398.prt,2,0012398.prt,0012398.prt,0012398.prt,,0012398.prt,2,0-02.0,Released,,nan,nan,nan,nan,nan,nan


In [250]:
import importlib

importlib.reload(id_changes)

reordered_nodes = id_changes.get_reordered_nodes(master_df, compared_df)

In [248]:
id_changes.get_added_nodes(master_df, compared_df)

['0015442.asm->0018435.asm->0019012.asm',
 '0015442.asm->0018435.asm->0019012.asm->0018439_revised_02.prt',
 '0015442.asm->0018435.asm->0019012.asm->0019011.prt',
 '0015442.asm->0018435.asm->0019012.asm->0007471.prt',
 '0015442.asm->0018435.asm->0019012.asm->0012468.prt',
 '0015442.asm->0012397.asm',
 '0015442.asm->0012397.asm->0012398.prt']

In [249]:
master_df.loc[(slice(None), id_changes.get_added_nodes(master_df, compared_df)), :]

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,AddedColumn1,AddedColumn2,AddedColumn3,AddedColumn4,AddedColumn5,AddedColumn6
Order,Node ID,,,,,,,,,,,,,,,


In [204]:
added_nodes_2 = id_changes.get_added_nodes(master_df, compared_df)

In [205]:
added_nodes_2

['0015442.asm->0018435.asm->0019012.asm',
 '0015442.asm->0018435.asm->0019012.asm->0018439_revised_02.prt',
 '0015442.asm->0018435.asm->0019012.asm->0019011.prt',
 '0015442.asm->0018435.asm->0019012.asm->0007471.prt',
 '0015442.asm->0018435.asm->0019012.asm->0012468.prt',
 '0015442.asm->0012397.asm',
 '0015442.asm->0012397.asm->0012398.prt']

In [214]:
master_df.loc[(slice(None), added_nodes_2), :]

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,AddedColumn1,AddedColumn2,AddedColumn3,AddedColumn4,AddedColumn5,AddedColumn6
Order,Node ID,,,,,,,,,,,,,,,


In [215]:
deleted_nodes_2 = id_changes.get_deleted_nodes(master_df, compared_df)

In [216]:
deleted_nodes_2

['0015442.asm->0018435.asm->0018442.asm->0018541.prt']

In [218]:
compared_df.loc[(slice(None), deleted_nodes_2), :]

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,TestOnly
Order,Node ID,,,,,,,,,,


In [243]:
import importlib

importlib.reload(id_changes)

id_changes.get_reordered_nodes(master_df, compared_df)

['0015442.asm->0018435.asm',
 '0015442.asm->0018435.asm->0018442.asm',
 '0015442.asm->0018435.asm->0018442.asm->0018441.prt',
 '0015442.asm->0018435.asm->0018442.asm->0012468.prt',
 '0015442.asm->0018435.asm->0018436.prt',
 '0015442.asm->0018435.asm->0018437.prt']

In [244]:
reordered_df = id_changes.get_reordered_idx(master_df, compared_df)

In [245]:
master_df[~deleted_nodes][reordered_df]

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,AddedColumn1,AddedColumn2,AddedColumn3,AddedColumn4,AddedColumn5,AddedColumn6
Order,Node ID,,,,,,,,,,,,,,,
2,0015442.asm->0018435.asm,1,0018435.asm,FS-8100 Chassis assembly - full,FS-8100 Chassis assembly - full,,FS-8100 Chassis assembly - full,1,0-00.24,Processing,,,,,,
3,0015442.asm->0018435.asm->0018442.asm,2,0018442.asm,FS-8100 Chassis Base Assembly,FS-8100 Chassis Base Assembly,,FS-8100 Chassis Base Assembly,1,0-00.6,Processing,,,,,,
4,0015442.asm->0018435.asm->0018442.asm->0018441.prt,3,0018441.prt,FS-9100 base blate - btm plate,FS-9100 base blate - btm plate,,FS-9100 base blate - btm plate,1,0-00.7,Processing,,,,,,
8,0015442.asm->0018435.asm->0018442.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,,RIVET,4,0-00.2,Released,,,,,,
9,0015442.asm->0018435.asm->0018436.prt,2,0018436.prt,FS-8100 Chassis vert member SM-Box,FS-8100 Chassis vert member SM-Box,,FS-8100 Chassis vert member SM-Box,4,0-00.15,Processing,,,,,,
10,0015442.asm->0018435.asm->0018437.prt,2,0018437.prt,0018437.prt,0018437.prt,,0018437.prt,1,0-00.9,Processing,,,,,,


In [253]:
def 

True

In [259]:
master_df[~deleted_nodes][updated_elements.apply(any, axis=1).values].reset_index()['Node ID'].to_list()

['0015442.asm->0018435.asm']

AttributeError: module 'id_changes' has no attribute 'get_added_columns_idx'